# PyTest: Unit Testing

Due to a class on testing, we won't cover much, just a few tips. I highly recommend taking some time to learn advanced PyTest, as anything that makes writing tests easiser enables more and better testing, which is always a plus!

## Tests should be easy

Always use PyTest. The built-in unittest is _very_ verbose; the simpler the writing of tests, the more tests you will write!

```python
import unittest

class MyTestCase(unittest.TestCase)
    def test_something(self):
        x = 1
        self.assertEquals(x, 2)
```

Contrast this to pytest:

```python
def test_something():
    x = 1
    assert x == 2
```

PyTest still gives you clear breakdowns, including what the value of `x` actually is, even though it seems to use the Python `assert` statement! You don't need to set up a class (though you can), and you don't need to remember 50 or so different `self.assert*` functions! PyTest can also run unittest tests, as well as the old `nose` package's tests, too.

Approximately equals is normally ugly to check, but PyTest makes it easy too:

```python
from pytest import approx

def test_approx():
    .3333333333333 == approx(1/3)
```

## Tests should test for failures too


You should make sure that expected errors are thrown:


```python
import pytest

def test_raises():
    with pytest.raises(ZeroDivisionError):
        1 / 0
```

You can check for warnings as well, with `pytest.warns` or `pytest.deprecated_call`.

## Tests should stay easy when scaling out

PyTest [uses fixtures](https://docs.pytest.org/en/stable/fixture.html) to represent complex ideas, like setup/teardown, temporary resources, or parameterization.


A fixture looks like a function argument; PyTest recognizes them by name:

```python
def test_printout(capsys):
    print("hello")
    
    captured = capsys.readouterr()
    assert "hello" in captured.out
```

Making a new fixture is not too hard, and can be placed in the test file or in `conftest.py`:


```python
@pytest.fixture(params=[1,2,3], ids=["one", "two", "three"])
def ints(request):
    return request.param
```

We could have left off `ids`, but for complex inputs, this lets the tests have beautiful names.

Now, you can use it:

```python
def test_on_ints(ints):
    assert ints**2 == ints*ints
```

Now you will get three tests, `test_on_ints-one`, `test_on_ints-two`, and `test_on_ints-three`!

Fixtures can be scoped, allowing simple setup/teardown (use `yield` if you need to run teardown). You can even set `autouse=True` to use a fixture always in a file or module (via `conftest.py`). You can have `conftest.py`'s in nested folders, too!

Here's an advanced example, which also uses `monkeypatch`, which is a great way for making things hard to split into units into unit tests. Let's say you wanted to make a test that "tricked" your code into thinking that it was running on different platforms:

```python
import platform
import pytest

@pytest.fixture(params=['Linux', 'Darwin', 'Windows'], autouse=True)
def platform_system(request, monkeypatch):
    monkeypatch.setattr(platform, "system", lambda _: request.param)
```

Now every test in the file this is in (or the directory that this is in if in conftest) will run three times, and each time will identify as a different `platform.system()`! Leave `autouse` off, and it becomes opt-in; adding `platform_system` to the list of arguments will opt in.

## Tests should be organized

You can use `pytest.mark.*` to mark tests, so you can easily turn on or off groups of tests, or do something else special with marked tests, like tests marked "slow", for example. Probably the most useful built-in mark is `skipif`:

```python
@pytest.mark.skipif("sys.version_info < (3, 8)")
def test_only_on_37plus():
    x = 3
    assert f"{x = }" == "x = 3"
```

Now this test will only run on Python 3.8, and will be skipped on earlier versions. You don't have to use a string for the condition, but if you don't, add a `reason=` so there will still be nice printout explaining why the test was skipped.

You can also use `xfail` for tests that are expected to fail (you can even strictly test them as failing if you want). You can use `parametrize` to make a single parameterized test instead of sharing them (with fixtures). There's a `filterwarnings` mark, too.

Many PyTest plugins support new marks too, like `pytest-parametrize`. You can also use custom marks to enable/disable groups of tests, or to pass data into fixtures.

## Tests should test the installed version, not the local version

Your tests should run against an _installed_ version of your code. Testing against the _local_ version might work with the installed version does not (due to a missing file, changed paths, etc). This is one of the big reasons to use `/src/package` instead of `/package`, as `python -m pytest` will pick up local directories and `pytest` does not.